In [1]:
import cv2
import numpy as np

In [2]:
# img_path = "3.2 aadhar_demo2"
img = cv2.imread("www.jpg")


In [3]:
#download the east model and load it 
model_path ="frozen_east_text_detection.pb"
model = cv2.dnn.readNet(model_path)

In [4]:
# ## Prepare the image
# use multiple of 32 to set the new img shape
height, width, _ = img.shape
new_height = (height//32)*32
new_width = (width//32)*32
print(new_height, new_width)


3104 4160


In [5]:
# get the ratio change in width and height
h_ratio = height/new_height
w_ratio = width/new_width
print(h_ratio, w_ratio)


1.0051546391752577 1.0


In [6]:
blob = cv2.dnn.blobFromImage(img, 1, (new_width, new_height),(123.68, 116.78, 103.94), True, False)

In [7]:
### pass the image to network and extract score and geometry map

model.setInput(blob)

In [8]:
model.getUnconnectedOutLayersNames()

['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3']

In [9]:
(geometry,scores) = model.forward(model.getUnconnectedOutLayersNames())

In [15]:
geometry.shape

(1, 1, 776, 1040)

In [16]:
### post-processing

rectangles = []
confidence_score = []
for i in range(geometry.shape[2]):
    for j in range(0,geometry.shape[3]):
        
        if scores[0][0][i][j] < 0.1:
            
            continue
            
        bottom_x = int(j*4 + geometry[0][0][i][j])
        bottom_y = int(i*4 + geometry[0][0][i][j])
        

        top_x = int(j*4 - geometry[0][0][i][j])
        top_y = int(i*4 - geometry[0][0][i][j])
        
        rectangles.append((top_x, top_y, bottom_x, bottom_y))
        confidence_score.append(float(scores[0][0][i][j]))


In [17]:
# pip install imutils

In [18]:
from imutils.object_detection  import non_max_suppression

In [19]:
# use non-max suppression to get the required rectangles

fin_boxes = non_max_suppression(np.array(rectangles),probs = confidence_score, overlapThresh = 0.5)

KeyboardInterrupt: 

In [ ]:
img_copy = img.copy()
for (x1,y1,x2,y2) in fin_boxes:
    
    x1 = int(x1 * w_ratio)
    y1 = int(y1 * h_ratio)
    x2 = int(x2 * w_ratio)
    y2 = int(y2 * h_ratio)
    
    cv2.rectangle(img_copy, (x1,y1),(x2,y2),(0,255,0),2)

In [ ]:
cv2.imshow("original Image", img)
cv2.imshow("Text Detection", img_copy)
cv2.waitKey(0)